In [3]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... one
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.3/782.3 kB 3.2 MB/s eta 0:00:00m-:--:--
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-3.0.11-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 34.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 27.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 42.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import parquet
from unidecode import unidecode
import zipfile
import random
from transformers import BertTokenizer, BertConfig
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import torch
from torch.utils.data import DataLoader, TensorDataset
import warnings

from src.features.functions_preprocessing import (
    plot_text_length_distribution,
    preprocess_articles,
    preprocess_summaries,
)
from src.features.tokenization import parallel_tokenize
from src.models.rnn_encoder_decoder import *
from src.models.transformer import Transformer
from src.models.bert import BertSummary

random.seed(42)

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
def get_allowed_cpu_count():
    # Returns the number of CPU cores available for this process.
    try:
        return len(os.sched_getaffinity(0))
    except AttributeError:
        return os.cpu_count() or 1


cpu_count = get_allowed_cpu_count()
print(cpu_count)

104


In [7]:
n_process = max(1, cpu_count // 2)

In [8]:
torch.set_num_threads(n_process)

# **Kaggle dataset**

In [ ]:
!kaggle datasets download -d sbhatti/news-summarization

In [ ]:
with zipfile.ZipFile("news-summarization.zip", "r") as zip_ref:
    zip_ref.extractall("news-summarization")

In [ ]:
news_data = pd.read_csv("news-summarization/data.csv")

In [ ]:
news_data.head()

In [ ]:
N = random.randint(1, len(news_data))

print(news_data["Content"][N])
print()
print(news_data["Summary"][N])

In [ ]:
lengths_article = news_data["Content"].str.len()
lengths_article.describe()

In [ ]:
news_data = news_data[
    (lengths_article >= lengths_article.quantile(0.10))
    & (lengths_article <= lengths_article.quantile(0.90))
]

In [ ]:
plot_text_length_distribution(news_data, "Content")

In [ ]:
lengths_summary = news_data["Summary"].str.len()
lengths_summary.describe()

In [ ]:
news_data = news_data[
    (lengths_summary >= lengths_summary.quantile(0.10))
    & (lengths_summary <= lengths_summary.quantile(0.90))
]

In [ ]:
news_data["Summary"].str.len().describe()

In [ ]:
plot_text_length_distribution(news_data, "Summary")

In [ ]:
len(news_data)

In [ ]:
# news_data.loc[:, "Content"] = preprocess_articles(
#     news_data["Content"].tolist(), n_process=n_process, batch_size=32
# )
# news_data.loc[:, "Summary"] = preprocess_summaries(
#     news_data["Summary"].tolist(), n_process=n_process, batch_size=32
# )

In [ ]:
# news_data.to_parquet("news_data_cleaned.parquet", index=False)

In [6]:
news_data = pd.read_parquet("news_data_cleaned.parquet")

# **Tokenization**

In [7]:
data_copy = news_data[:]
data_copy = news_data.sample(frac=1, random_state=42)

train_ratio = 0.8
train_size = int(train_ratio * len(data_copy))

# Slice the dataset
train_data = data_copy[:train_size]
test_data = data_copy[train_size:]

print(f"Train size: {len(train_data)}")
print(f"Test size:  {len(test_data)}")

Train size: 446425
Test size:  111607


In [18]:
# if __name__ == "__main__":
#     texts_content = list(train_data["Content"])

#     print("Tokenizing Content...")
#     tokenized_articles = parallel_tokenize(
#         texts_content,
#         tokenizer_name="bert-base-uncased",
#         max_workers=n_process,
#         chunk_size=2000,
#         max_length=512,
#     )
#     print("tokenized_articles.shape =", tokenized_articles.shape)
#     torch.save(tokenized_articles, "tokenized_articles.pt")

Tokenizing Content...
tokenized_articles.shape = torch.Size([446425, 512])
Tokenizing Summaries...



KeyboardInterrupt



In [8]:
# if __name__ == "__main__":
#     texts_summary = list(train_data["Summary"])

#     print("Tokenizing Summaries...")
#     tokenized_summaries = parallel_tokenize(
#         texts_summary,
#         tokenizer_name="bert-base-uncased",
#         max_workers=n_process,
#         chunk_size=2000,
#         max_length=512,
#     )
#     print("tokenized_summaries.shape =", tokenized_summaries.shape)
#     torch.save(tokenized_summaries, "tokenized_summaries.pt")

Tokenizing Summaries...
tokenized_summaries.shape = torch.Size([446425, 512])


In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tokenized_articles = torch.load("tokenized_articles.pt")
    tokenized_summaries = torch.load("tokenized_summaries.pt")

In [10]:
tokenized_articles = tokenized_articles.long()

tokenized_summaries = torch.cat(
    [torch.zeros(tokenized_summaries.size(0), 1), tokenized_summaries], dim=1
).long()

article_ids = tokenized_articles.long()
summary_ids = tokenized_summaries.long()

# **RNN**

# **Transformer**

In [16]:
batch_size = 10

dataset = TensorDataset(tokenized_articles, tokenized_summaries)
dataloader = DataLoader(
    dataset, batch_size=batch_size, num_workers=n_process, shuffle=True
)

In [17]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [18]:
model_transformer = Transformer(
    pad_idx=0,
    voc_size=tokenizer.vocab_size,
    hidden_size=128,
    n_head=8,
    max_len=512,
    dec_max_len=512,
    ffn_hidden=128,
    n_layers=3,
)

In [ ]:
num_epochs = 50

loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model_transformer.parameters(), lr=5e-4)
model = model_transformer.to(device)

total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    model.train()
    total_loss = 0

    for step, batch in enumerate(dataloader):
        input_batch, summary_batch = batch
        input_batch = input_batch.to(device)
        summary_batch = summary_batch.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_batch.long(), summary_batch[:, :-1])

        # Shift the target by one for the loss
        summary_batch = summary_batch[:, 1:]

        # Compute loss
        loss = loss_fn(
            outputs.reshape(-1, outputs.shape[-1]), summary_batch.reshape(-1)
        )
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        if step % 1000 == 0:
            print(f"Step-{step}, Loss-{loss.item():.4f}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(dataloader)

    # Calculate epoch duration
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time

    # Print epoch stats
    print(
        f"Epoch {epoch+1}/{num_epochs} - "
        f"Average Loss: {avg_loss:.4f} - "
        f"Time: {epoch_duration:.2f}s"
    )

# Calculate total training time
total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f"Total training time: {total_training_time:.2f}s")

Step-0, Loss-10.487357139587402


# **BERT model**

In [11]:
batch_size = 10

dataset = TensorDataset(tokenized_articles, tokenized_summaries[:, 1:])
dataloader = DataLoader(
    dataset, batch_size=batch_size, num_workers=n_process, shuffle=True
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [14]:
config = BertConfig(
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    vocab_size=tokenizer.vocab_size,
)

In [15]:
model = BertSummary(config)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
model.to(device)

num_epochs = 2

total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    model.train()
    total_loss = 0

    for step, batch in enumerate(dataloader):
        input_batch, summary_batch = batch
        input_batch = input_batch.to(device)
        summary_batch = summary_batch.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_batch, attention_mask=input_batch.ne(0))

        # Compute loss
        loss = loss_fn(outputs.view(-1, outputs.shape[-1]), summary_batch.view(-1))
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        if step % 1000 == 0:
            print(f"Epoch: {epoch+1}, Step: {step}, Loss: {loss.item():.4f}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(dataloader)

    # Measure epoch time
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time

    print(
        f"Epoch {epoch+1}/{num_epochs} - "
        f"Average Loss: {avg_loss:.4f} - "
        f"Time: {epoch_duration:.2f}s"
    )

# Measure total training time
total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f"Total training time: {total_training_time:.2f}s")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Step-0, Loss-10.3932523727417


KeyboardInterrupt: 

# **Evaluation**

In [ ]:
!pip install

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
input_text = news_data["Content"][1000]

In [ ]:
tokenized_input = tokenizer.encode_plus(
    input_text,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_tensors="pt",
)

In [ ]:
input_ids = tokenized_input["input_ids"].to(device)
attention_mask = tokenized_input["attention_mask"].to(device)

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

In [ ]:
# Reshape the outputs tensor
reshaped_outputs = outputs

# Get the predicted summary
predicted_summary_ids = torch.argmax(reshaped_outputs, dim=-1)
predicted_summary = tokenizer.decode(predicted_summary_ids[0], skip_special_tokens=True)

print("Predicted Summary:", predicted_summary)